In [8]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import os
from google.colab import drive
drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
class EmotionClassifier(nn.Module):
    def __init__(self, num_labels=28):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = output.last_hidden_state[:, 0]
        pooled = self.dropout(pooled)
        return self.classifier(pooled)


In [10]:
# Paths
drive_path = "/content/drive/MyDrive/Project/models"
general_model_paths = [f"{drive_path}/bert_fold_{i}.pt" for i in range(1, 6)]
rare_model_path = f"{drive_path}/rare_emotion_model.pt"

# Loading the models
general_models = []
for path in general_model_paths:
    model = EmotionClassifier(num_labels=28).to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()
    general_models.append(model)

# Loading rare model
rare_model = EmotionClassifier(num_labels=12).to(device)
rare_model.load_state_dict(torch.load(rare_model_path, map_location=device))
rare_model.eval()


EmotionClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
import os
from tqdm import tqdm

# Defining Rare emotions
RARE_EMOTIONS = [
    'grief', 'relief', 'curiosity', 'realization', 'pride', 'nervousness',
    'confusion', 'caring', 'disappointment', 'annoyance', 'approval', 'disapproval'
]

ALL_EMOTIONS = [
    "admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion", "curiosity", "desire",
    "disappointment", "disapproval", "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief", "joy",
    "love", "nervousness", "optimism", "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral"
]

rare_indices = [ALL_EMOTIONS.index(e) for e in RARE_EMOTIONS]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_dir = "/content/drive/MyDrive/Project/models"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def load_general_model(path):
    model = EmotionClassifier(num_labels=28)  # matches how you trained/saved
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    model.eval()
    return model

general_models = [load_general_model(f"{model_dir}/bert_fold_{i}.pt") for i in range(1, 6)]

class RareEmotionClassifier(torch.nn.Module):
    def __init__(self, num_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return self.classifier(self.dropout(outputs.pooler_output))

rare_model = RareEmotionClassifier(num_labels=len(RARE_EMOTIONS))
rare_model.load_state_dict(torch.load(f"{model_dir}/rare_emotion_model.pt", map_location=device))
rare_model.to(device)
rare_model.eval()

def encode_batch(texts, max_len=128):
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt", max_length=max_len)
    return {k: v.to(device) for k, v in encodings.items()}

# Ensemble prediction function
def ensemble_predict(texts, alpha=0.5):
    inputs = encode_batch(texts)

    # General logits from all 5 models
    general_logits = [model(inputs["input_ids"], inputs["attention_mask"]) for model in general_models]
    general_logits = torch.stack(general_logits).mean(dim=0)  # shape: [B, 28]

    # Rare logits (12)
    rare_logits = rare_model(inputs["input_ids"], inputs["attention_mask"])  # shape: [B, 12]

    # Blending general + rare logits for rare indices
    final_logits = general_logits.clone()
    for i, rare_idx in enumerate(rare_indices):
        final_logits[:, rare_idx] = (1 - alpha) * general_logits[:, rare_idx] + alpha * rare_logits[:, i]

    probs = torch.sigmoid(final_logits)

    # Updated per-label thresholds 
    thresholds = torch.tensor([
        0.60, 0.60, 0.55, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60,
        0.60, 0.60, 0.65, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60,
        0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60, 0.60
    ]).to(probs.device)

    preds = (probs > thresholds).int()

    return preds.detach().cpu().numpy(), probs.detach().cpu().numpy()


In [26]:
def print_ensemble_results(texts, preds, probs, thresholds, top_k=3):
    for i, text in enumerate(texts):
        print(f"\nText: {text}\n")
        count = 0
        for j in range(len(ALL_EMOTIONS)):
            if probs[i][j] >= thresholds[j]:
                print(f"  {ALL_EMOTIONS[j]:<15}: {probs[i][j]:.3f} ")
                count += 1

        if count == 0:
            print(f"  No emotions above threshold! Showing top {top_k} anyway:")
            top_indices = probs[i].argsort()[-top_k:][::-1]
            for idx in top_indices:
                print(f"  {ALL_EMOTIONS[idx]:<15}: {probs[i][idx]:.3f} ")

# Thresholds
thresholds = torch.tensor([
    max(0.5, t) for t in [
        0.11, 0.41, 0.22, 0.15, 0.10, 0.10, 0.11, 0.18, 0.10, 0.10,
        0.21, 0.12, 0.60, 0.14, 0.10, 0.60, 0.10, 0.11, 0.10, 0.10,
        0.18, 0.28, 0.13, 0.14, 0.10, 0.11, 0.10, 0.16
    ]
]).to(probs.device)

texts = ["I feel completely hopeless and broken", "I'm so proud of my achievement!"]
preds, probs = ensemble_predict(texts)
print_ensemble_results(texts, preds, probs,thresholds)



Text: I feel completely hopeless and broken

  admiration     : 0.520 
  amusement      : 0.557 
  anger          : 0.585 
  annoyance      : 0.516 
  approval       : 0.505 
  caring         : 0.531 
  curiosity      : 0.559 
  disgust        : 0.500 
  fear           : 0.537 
  grief          : 0.525 
  joy            : 0.511 
  nervousness    : 0.511 
  optimism       : 0.523 
  pride          : 0.545 
  relief         : 0.507 
  remorse        : 0.505 

Text: I'm so proud of my achievement!

  admiration     : 0.501 
  caring         : 0.517 
  curiosity      : 0.516 
  desire         : 0.523 
  disappointment : 0.541 
  love           : 0.535 
  nervousness    : 0.508 
  optimism       : 0.560 
